In [ ]:
!pip install qdrant-haystack
!pip install fastembed

# Scraping Documentation

In [1]:
import requests
from bs4 import BeautifulSoup
import os

def get_sitemap_data(url):
    """
    Retrieves the sitemap.xml data from the given URL.

    Args:
        url (str): The base URL of the documentation website.

    Returns:
        str: The content of the sitemap.xml file.
    """

    sitemap_url = f"{url}/sitemap.xml"
    try:
        response = requests.get(sitemap_url)
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching sitemap: {e}")
        return None

def extract_urls_from_sitemap(sitemap_data):
    """
    Extracts URLs from the given sitemap.xml data.

    Args:
        sitemap_data (str): The content of the sitemap.xml file.

    Returns:
        list: A list of URLs extracted from the sitemap.
    """

    soup = BeautifulSoup(sitemap_data, 'xml')
    urls = []
    for url_tag in soup.find_all('url'):
        loc_tag = url_tag.find('loc')
        if loc_tag:
            urls.append(loc_tag.text)
    return urls

def fetch_and_store_documentation(base_url):
    """
    Fetches documentation content from URLs and stores them in a dictionary.

    Args:
        base_url (str): The base URL of the documentation website.

    Returns:
        dict: A dictionary where keys are URLs and values are filtered HTML content.
    """

    sitemap_data = get_sitemap_data(base_url)
    if sitemap_data:
        urls = extract_urls_from_sitemap(sitemap_data)
        docs = {}  # Initialize an empty dictionary

        for url in urls:
            try:
                response = requests.get(url)
                response.raise_for_status()

                soup = BeautifulSoup(response.text, 'html.parser')
                # Filter out unwanted tags using BeautifulSoup (adjust as needed)
                for tag in ['script', 'style', 'nav', 'aside', 'footer']:
                    for element in soup.find_all(tag):
                        element.decompose()

                docs[url] = soup.get_text(separator=' ')  # Store filtered HTML content
                print(f"Fetched and stored content from: {url}")

            except requests.exceptions.RequestException as e:
                print(f"Error fetching {url}: {e}")

        return docs
    else:
        return None

In [2]:
# Example usage:
base_url = "https://llama-cpp-python.readthedocs.io/"
documentation_data = fetch_and_store_documentation(base_url)

Fetched and stored content from: https://llama-cpp-python.readthedocs.io/en/stable/
Fetched and stored content from: https://llama-cpp-python.readthedocs.io/en/latest/


In [3]:
from nltk.tokenize import sent_tokenize

for url, content in documentation_data.items():
    sentences = sent_tokenize(content)
    documentation_data[url] = sentences
    print(f"Sentences in {url}: {len(sentences)}")

Sentences in https://llama-cpp-python.readthedocs.io/en/stable/: 144
Sentences in https://llama-cpp-python.readthedocs.io/en/latest/: 144


In [4]:
from fastembed import TextEmbedding
import numpy as np
import time

# Initialize the TextEmbedding model
embedding_model = TextEmbedding(model_name="BAAI/bge-small-en-v1.5", cache_dir="./embeddings")

def embed_documents(documents):
    for url, sentences in documentation_data.items():
        
        embeddings = []
        for sentence in sentences:
            # Embed document using FastEmbed
            embedding = np.array(list((embedding_model.embed([sentence]))))
            
            # Append the embedding to the list of embeddings
            embeddings.append((sentence,embedding))
        
        documentation_data[url] = embeddings
        
    return documentation_data

# Perform embedding generation
documentation_data = embed_documents(documentation_data)

/home/yash/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 110960.42it/s]


In [23]:
from haystack.dataclasses.document import Document
from haystack_integrations.document_stores.qdrant import QdrantDocumentStore

ingestion_data = []

document_store = QdrantDocumentStore(
    ":memory:",
    index="Document",
    embedding_dim=384,
    recreate_index=True,
    hnsw_config={"m": 16, "ef_construct": 64}  # Optional
)

for url, sentences in documentation_data.items():
    # print(sentences[0][0])
    ingestion_data.append(Document(content=sentences[0][0], embedding=sentences[0][1][0], meta={"url": url}))
    try:
        document_store.write_documents(ingestion_data)
    except:
        # Duplicate document
        pass


100it [00:00, 45427.32it/s]          


In [24]:
print(document_store.count_documents())

1


In [ ]:
from haystack_integrations.components.retrievers.qdrant import QdrantEmbeddingRetriever

retriever = QdrantEmbeddingRetriever(document_store=document_store)

query = "How to install Llama-cpp ?"


retriever.run(query_embedding=[0.1]*768)